<a href="https://colab.research.google.com/github/PhamBaVuong/PhamBaVuong_DATN/blob/main/Routing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install polyline

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import requests
import folium
import polyline

In [3]:
url = "http://router.project-osrm.org/route/v1/driving/13.388860,52.517037;13.397634,52.529407;13.428555,52.523219?overview=false"
r = requests.get(url)
res = r.json()
res

{'code': 'Ok',
 'routes': [{'legs': [{'steps': [],
     'summary': '',
     'weight': 260.8,
     'duration': 259.6,
     'distance': 1884},
    {'steps': [],
     'summary': '',
     'weight': 365.7,
     'duration': 365.7,
     'distance': 2839.2}],
   'weight_name': 'routability',
   'weight': 626.5,
   'duration': 625.3,
   'distance': 4723.2}],
 'waypoints': [{'hint': 'hAEKgEwlmYUXAAAABQAAAAAAAAAgAAAAfXRPQdLNK0AAAAAAsPePQQsAAAADAAAAAAAAABAAAAAb7wAA_kvMAKlYIQM8TMwArVghAwAA7wobj63C',
   'distance': 4.231666,
   'name': 'Friedrichstraße',
   'location': [13.388798, 52.517033]},
  {'hint': 'g_jcgWl6iIcGAAAACgAAAAAAAAB3AAAA5JSNQJ0fwkAAAAAAGjmEQgYAAAAKAAAAAAAAAHcAAAAb7wAAfm7MABiJIQOCbswA_4ghAwAAXwUbj63C',
   'distance': 2.795167,
   'name': 'Torstraße',
   'location': [13.39763, 52.529432]},
  {'hint': 'ZywYgP___38fAAAAUQAAACYAAAAeAAAAsowKQkpQX0Lf6yZCvsQGQh8AAABRAAAAJgAAAB4AAAAb7wAASufMAOdwIQNL58wA03AhAwMAvxAbj63C',
   'distance': 2.226595,
   'name': 'Platz der Vereinten Nationen',
   

In [8]:
def get_route(pickup_lon, pickup_lat, dropoff_lon, dropoff_lat):
    
    loc = "{},{};{},{}".format(pickup_lon, pickup_lat, dropoff_lon, dropoff_lat)
    url = "http://router.project-osrm.org/route/v1/driving/"
    r = requests.get(url + loc) 
    if r.status_code!= 200:
        return {}
  
    res = r.json()   
    routes = polyline.decode(res['routes'][0]['geometry'])
    start_point = [res['waypoints'][0]['location'][1], res['waypoints'][0]['location'][0]]
    end_point = [res['waypoints'][1]['location'][1], res['waypoints'][1]['location'][0]]
    distance = res['routes'][0]['distance']
    
    out = {'route':routes,
           'start_point':start_point,
           'end_point':end_point,
           'distance':distance
          }

    return out

In [17]:
pickup_lat, pickup_lon, dropoff_lat, dropoff_lon = 10.851114146868106,106.6638882232723 , 10.841712258705531,106.66482746593148
test_route = get_route(pickup_lon, pickup_lat, dropoff_lon, dropoff_lat)
test_route

{'route': [(10.85112, 106.66388),
  (10.85097, 106.66372),
  (10.85083, 106.66399),
  (10.8505, 106.66386),
  (10.84982, 106.66497),
  (10.84823, 106.6646),
  (10.84781, 106.66458),
  (10.84522, 106.66479),
  (10.8436, 106.66466),
  (10.84171, 106.66481)],
 'start_point': [10.851122, 106.663883],
 'end_point': [10.84171, 106.664806],
 'distance': 1152.5}

In [18]:
def get_map(route):
    
    m = folium.Map(location=[(route['start_point'][0] + route['end_point'][0])/2, 
                             (route['start_point'][1] + route['end_point'][1])/2], 
                   zoom_start=13)

    folium.PolyLine(
        route['route'],
        weight=8,
        color='blue',
        opacity=0.6
    ).add_to(m)

    folium.Marker(
        location=route['start_point'],
        icon=folium.Icon(icon='play', color='green')
    ).add_to(m)

    folium.Marker(
        location=route['end_point'],
        icon=folium.Icon(icon='stop', color='red')
    ).add_to(m)

    return m

In [19]:
get_map(test_route)

In [16]:
freegeoip = "http://freegeoip.net/json"
geo_r = requests.get(freegeoip)
geo_json = geo_r.json()

user_postition = [geo_json["latitude"], geo_json["longitude"]]

print(user_postition)

JSONDecodeError: ignored